# YOLOv8 Bone Fracture Detection - Model Training

This notebook trains a YOLOv8 model on the cleaned bone fracture detection dataset.

In [3]:
# Install ultralytics (YOLOv8)
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.4/810.4 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 8.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 MB 14.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [ultralytics] [ultralytics]
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import yaml
import torch

# Check what devices are available
print("Device Check:")
print(f"  CUDA available: {torch.cuda.is_available()}")
print(f"  MPS (Apple GPU) available: {torch.backends.mps.is_available() if hasattr(torch.backends, 'mps') else False}")
print(f"  CPU: Always available")

# Determine best device
if torch.cuda.is_available():
    device = 0
    device_name = "CUDA GPU"
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'  # Apple Silicon GPU
    device_name = "Apple MPS (GPU)"
else:
    device = 'cpu'
    device_name = "CPU"

print(f"\nUsing device: {device_name} ({device})")

Device Check:
  CUDA available: False
  MPS (Apple GPU) available: True
  CPU: Always available

Using device: Apple MPS (GPU) (mps)


In [12]:
# Set up paths
# Update this path to point to your cleaned dataset
data_yaml_path = "BoneFractureYolo8/data.yaml"

# Verify the data.yaml file exists and is correct
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)
    print("Dataset Configuration:")
    print(f"  Number of classes: {data_config['nc']}")
    print(f"  Class names: {data_config['names']}")
    print(f"  Train path: {data_config['train']}")
    print(f"  Val path: {data_config['val']}")
    print(f"  Test path: {data_config['test']}")

Dataset Configuration:
  Number of classes: 6
  Class names: ['elbow positive', 'fingers positive', 'forearm fracture', 'humerus fracture', 'shoulder fracture', 'wrist positive']
  Train path: ../train/images
  Val path: ../valid/images
  Test path: ../test/images


In [13]:
# Initialize YOLOv8 model
# Options: 'yolov8n.pt' (nano), 'yolov8s.pt' (small), 'yolov8m.pt' (medium), 
#          'yolov8l.pt' (large), 'yolov8x.pt' (xlarge)
# Start with 'yolov8n.pt' for faster training, or 'yolov8s.pt' for better accuracy
model = YOLO('yolov8n.pt')  # Change to yolov8s.pt, yolov8m.pt, etc. for better accuracy
print(f"Model initialized: {model.model_name}")

Model initialized: yolov8n.pt


In [14]:
# Train the model
# Device is automatically detected from Cell 2 (will use MPS/GPU if available, otherwise CPU)
results = model.train(
    data=data_yaml_path,
    epochs=100,              # Number of training epochs
    imgsz=640,              # Image size
    batch=16 if device != 'cpu' else 8,  # Larger batch for GPU, smaller for CPU
    name='bone_fracture_yolov8',  # Project name
    project='runs/detect',  # Project directory
    patience=20,            # Early stopping patience
    save=True,              # Save checkpoints
    plots=True,             # Generate training plots
    val=True,               # Validate during training
    device=device,          # Uses device detected in Cell 2 (MPS/GPU/CPU)
    workers=8 if device != 'cpu' else 4,  # More workers for GPU
)

Ultralytics 8.4.14 🚀 Python-3.13.5 torch-2.10.0 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=BoneFractureYolo8/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=bone_fracture_yolov8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0

## Training Results

After training, the model will be saved in `runs/detect/bone_fracture_yolov8/weights/best.pt`

In [15]:
# Load the best model from training
best_model_path = 'runs/detect/bone_fracture_yolov8/weights/best.pt'
if os.path.exists(best_model_path):
    model = YOLO(best_model_path)
    print(f"Loaded best model from: {best_model_path}")
else:
    print(f"Model not found at {best_model_path}. Training may still be in progress.")

Model not found at runs/detect/bone_fracture_yolov8/weights/best.pt. Training may still be in progress.


In [16]:
# Validate the model on validation set
metrics = model.val(data=data_yaml_path)
print("\nValidation Metrics:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

Ultralytics 8.4.14 🚀 Python-3.13.5 torch-2.10.0 CPU (Apple M4 Pro)
Model summary (fused): 73 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 94.0±13.2 MB/s, size: 11.6 KB)
val: Scanning /Users/matthewgerges/Documents/Waterloo/4A/MSE446/Term Project/boneFracture/BoneFractureYolo8/valid/labels.cache... 348 images, 175 backgrounds, 17 corrupt: 100% ━━━━━━━━━━━━ 348/348 26.5Mit/s 0.0s
val: /Users/matthewgerges/Documents/Waterloo/4A/MSE446/Term Project/boneFracture/BoneFractureYolo8/valid/images/image1_1090_png.rf.de645f822a5e36175c5e988223f4eeb0.jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 6. Possible class labels are 0-5
val: /Users/matthewgerges/Documents/Waterloo/4A/MSE446/Term Project/boneFracture/BoneFractureYolo8/valid/images/image1_111_png.rf.3893d8f7588cea4d796d26119e52637f.jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 6. Possible class labels are 0-5
val: /Users/m

In [17]:
# Run inference on a few test images to visualize predictions
import glob
import cv2

# Get a few test images
test_images_dir = "BoneFractureYolo8/test/images"
test_images = glob.glob(os.path.join(test_images_dir, "*.jpg"))[:5]  # Get first 5 images

# Run predictions
for img_path in test_images:
    results = model.predict(img_path, save=True, conf=0.25)
    print(f"Processed: {img_path}")


image 1/1 /Users/matthewgerges/Documents/Waterloo/4A/MSE446/Term Project/boneFracture/BoneFractureYolo8/test/images/image1_704_png.rf.e3668d404753b933e1fd9500e697ca8c.jpg: 512x640 (no detections), 56.9ms
Speed: 5.4ms preprocess, 56.9ms inference, 2.5ms postprocess per image at shape (1, 3, 512, 640)
Results saved to /Users/matthewgerges/Documents/Waterloo/4A/MSE446/Term Project/boneFracture/runs/detect/predict
Processed: BoneFractureYolo8/test/images/image1_704_png.rf.e3668d404753b933e1fd9500e697ca8c.jpg

image 1/1 /Users/matthewgerges/Documents/Waterloo/4A/MSE446/Term Project/boneFracture/BoneFractureYolo8/test/images/image1_1326_png.rf.6019c7e15df84d9d6286932d03e687b6.jpg: 320x640 (no detections), 28.7ms
Speed: 0.6ms preprocess, 28.7ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 640)
Results saved to /Users/matthewgerges/Documents/Waterloo/4A/MSE446/Term Project/boneFracture/runs/detect/predict
Processed: BoneFractureYolo8/test/images/image1_1326_png.rf.6019c7e15df84